In [1]:
import pandas as pd
import plotly.graph_objects as go
from collections import deque
from IPython.display import display, HTML

In [2]:
trades_wn_dn1 = pd.read_csv('trades_round_1_day_-1_wn.csv', delimiter=';')
trades_wn_dn2 = pd.read_csv('trades_round_1_day_-2_wn.csv', delimiter=';')
trades_wn_d0 = pd.read_csv('trades_round_1_day_0_wn.csv', delimiter=';')

trades_r3_d0 = pd.read_csv('trades_round_3_day_0_wn.csv', delimiter=';')
trades_r3_d1 = pd.read_csv('trades_round_3_day_1_wn.csv', delimiter=';')
trades_r3_d2 = pd.read_csv('trades_round_3_day_2_wn.csv', delimiter=';')

trades_r4_d1 = pd.read_csv('trades_round_4_day_1_wn.csv', delimiter=';')
trades_r4_d2 = pd.read_csv('trades_round_4_day_2_wn.csv', delimiter=';')
trades_r4_d3 = pd.read_csv('trades_round_4_day_3_wn.csv', delimiter=';')

In [3]:
prices_r1_dn1 = pd.read_csv('prices_round_1_day_-1.csv', delimiter=';')
prices_r1_dn2 = pd.read_csv('prices_round_1_day_-2.csv', delimiter=';')
prices_r1_d0 = pd.read_csv('prices_round_1_day_0.csv', delimiter=';')

prices_r3_d0 = pd.read_csv('prices_round_3_day_0.csv', delimiter=';')
prices_r3_d1 = pd.read_csv('prices_round_3_day_1.csv', delimiter=';')
prices_r3_d2 = pd.read_csv('prices_round_3_day_2.csv', delimiter=';')

prices_r4_d1 = pd.read_csv('prices_round_4_day_1.csv', delimiter=';')
prices_r4_d2 = pd.read_csv('prices_round_4_day_2.csv', delimiter=';')
prices_r4_d3 = pd.read_csv('prices_round_4_day_3.csv', delimiter=';')

In [4]:
prices_r1_d0

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
0,0,0,AMETHYSTS,9998,1,9995.0,30.0,NaN,NaN,10005,30,NaN,NaN,NaN,NaN,10001.5,0.0
1,0,0,STARFRUIT,5036,30,NaN,NaN,NaN,NaN,5043,30,NaN,NaN,NaN,NaN,5039.5,0.0
2,0,100,AMETHYSTS,9996,1,9995.0,30.0,NaN,NaN,10002,6,10004.0,1.0,10005.0,30.0,9999.0,0.0
3,0,100,STARFRUIT,5041,4,5037.0,1.0,5036.0,30.0,5043,31,NaN,NaN,NaN,NaN,5042.0,0.0
4,0,200,AMETHYSTS,9996,1,9995.0,21.0,NaN,NaN,10004,1,10005.0,21.0,NaN,NaN,10000.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0,999700,STARFRUIT,5053,1,5048.0,26.0,NaN,NaN,5054,2,5055.0,24.0,NaN,NaN,5053.5,0.0
19996,0,999800,STARFRUIT,5047,27,NaN,NaN,NaN,NaN,5052,1,5054.0,27.0,NaN,NaN,5049.5,0.0
19997,0,999800,AMETHYSTS,9995,27,NaN,NaN,NaN,NaN,10005,27,NaN,NaN,NaN,NaN,10000.0,0.0
19998,0,999900,STARFRUIT,5048,2,5047.0,21.0,NaN,NaN,5054,23,NaN,NaN,NaN,NaN,5051.0,0.0


In [5]:
def plot_trades(df_trades, df_prices, trader_name, product_name):
    buys = df_trades[(df_trades['buyer'] == trader_name) & (df_trades['symbol'] == product_name)]
    sells = df_trades[(df_trades['seller'] == trader_name) & (df_trades['symbol'] == product_name)]
    mid_prices = df_prices[df_prices['product'] == product_name]

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=buys['timestamp'], y=buys['price'], mode='markers', name='Buys', marker=dict(color='green', size=10, opacity=0.5)))
    fig.add_trace(go.Scatter(x=sells['timestamp'], y=sells['price'], mode='markers', name='Sells', marker=dict(color='red', size=10, opacity=0.5)))
    fig.add_trace(go.Scatter(x=mid_prices['timestamp'], y=mid_prices['mid_price'], mode='lines', name='Mid Price', line=dict(color='blue', width=2)))

    fig.update_layout(title=f'Trades for {trader_name} on {product_name}',
                      xaxis_title='Timestamp',
                      yaxis_title='Price',
                      legend_title='Transaction Type')
    fig.show()

In [6]:
def plot_realized_pl(df, trader_name, product_name):
    buys = df[(df['buyer'] == trader_name) & (df['symbol'] == product_name)].copy()
    buys['type'] = 'buy'
    buys['quantity'] = buys['quantity']
    
    sells = df[(df['seller'] == trader_name) & (df['symbol'] == product_name)].copy()
    sells['type'] = 'sell'
    sells['quantity'] = -sells['quantity']  # Treat sells as negative for uniform handling
    
    transactions = pd.concat([buys, sells])
    transactions.sort_values('timestamp', inplace=True)

    positions = deque()
    realized_pl = []
    pl_total = 0

    for _, transaction in transactions.iterrows():
        transaction_quantity = transaction['quantity']
        transaction_price = transaction['price']
        transaction_timestamp = transaction['timestamp']

        while transaction_quantity != 0:
            if not positions:
                positions.append({'quantity': transaction_quantity, 'price': transaction_price})
                break

            position = positions[0]
            if (position['quantity'] > 0 and transaction_quantity < 0) or (position['quantity'] < 0 and transaction_quantity > 0):
                quantity_to_close = min(abs(transaction_quantity), abs(position['quantity']))
                pl = quantity_to_close * (abs(transaction_price) - abs(position['price']))
                if position['quantity'] < 0:
                    pl = -pl

                pl_total += pl
                realized_pl.append({'timestamp': transaction_timestamp, 'P&L': pl_total})

                position['quantity'] += transaction_quantity
                if position['quantity'] == 0:
                    positions.popleft()  # Remove closed position
                transaction_quantity += quantity_to_close if transaction_quantity < 0 else -quantity_to_close
            else:
                positions.append({'quantity': transaction_quantity, 'price': transaction_price})
                break

    pl_df = pd.DataFrame(realized_pl)
    if not pl_df.empty:
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=pl_df['timestamp'], y=pl_df['P&L'], mode='lines+markers', name='Realized P&L'))
        fig.update_layout(title=f'Realized P&L for {trader_name} on {product_name}',
                          xaxis_title='Timestamp',
                          yaxis_title='Profit and Loss',
                          legend_title='P&L')
        fig.show()
    else:
        print("No completed transactions to calculate realized P&L.")


In [7]:
def analyze_trades(df_trades, df_prices):
    def get_unique_symbols(df):
        return df['symbol'].unique()
    
    def get_unique_traders(df):
        buyers = df['buyer'].unique()
        sellers = df['seller'].unique()
        return set(buyers).union(set(sellers))
    
    def get_buys(df, trader_name):
        return df[df['buyer'] == trader_name]

    def get_sells(df, trader_name):
        return df[df['seller'] == trader_name]

    unique_traders = get_unique_traders(df_trades)
    unique_symbols = get_unique_symbols(df_trades)

    for trader in unique_traders:
        for symbol in unique_symbols:
            print(f"\nTrader: {trader}, Symbol: {symbol}")
            buys = get_buys(df_trades, trader)
            sells = get_sells(df_trades, trader)

            buys = buys[buys['symbol'] == symbol]
            sells = sells[sells['symbol'] == symbol]

            if not buys.empty:
                print("Buys:")
                display(buys)
            else:
                print("No buys for this product.")

            if not sells.empty:
                print("Sells:")
                display(sells)
            else:
                print("No sells for this product.")

            if not buys.empty or not sells.empty:
                plot_trades(df_trades, df_prices, trader, symbol)

            if not buys.empty and not sells.empty:
                plot_realized_pl(df_trades, trader, symbol)
            else:
                print("Not enough data to calculate realized P&L.")


In [8]:
# copypaste into the cell below to see 1 at a time
"""
analyze_trades(trades_wn_dn2, prices_r1_dn2)
analyze_trades(trades_wn_dn1, prices_r1_dn1)
analyze_trades(trades_wn_d0, prices_r1_d0)

analyze_trades(trades_r3_d0, prices_r3_d0)
analyze_trades(trades_r3_d1, prices_r3_d1)
analyze_trades(trades_r3_d2, prices_r3_d2)

analyze_trades(trades_r4_d1, prices_r4_d1)
analyze_trades(trades_r4_d2, prices_r4_d2)
analyze_trades(trades_r4_d3, prices_r4_d3)
"""

'\nanalyze_trades(trades_wn_dn2, prices_r1_dn2)\nanalyze_trades(trades_wn_dn1, prices_r1_dn1)\nanalyze_trades(trades_wn_d0, prices_r1_d0)\n\nanalyze_trades(trades_r3_d0, prices_r3_d0)\nanalyze_trades(trades_r3_d1, prices_r3_d1)\nanalyze_trades(trades_r3_d2, prices_r3_d2)\n\nanalyze_trades(trades_r4_d1, prices_r4_d1)\nanalyze_trades(trades_r4_d2, prices_r4_d2)\nanalyze_trades(trades_r4_d3, prices_r4_d3)\n'

In [9]:
analyze_trades(trades_r4_d1, prices_r4_d1)


Trader: Raj, Symbol: COCONUT_COUPON
No buys for this product.
No sells for this product.
Not enough data to calculate realized P&L.

Trader: Raj, Symbol: COCONUT
Buys:


,timestamp,buyer,seller,symbol,currency,price,quantity
18,9300,Raj,Vinnie,COCONUT,SEASHELLS,10005.0,6
76,43100,Raj,Vinnie,COCONUT,SEASHELLS,9974.0,8
90,48100,Raj,Vinnie,COCONUT,SEASHELLS,9969.0,8
133,70300,Raj,Vinnie,COCONUT,SEASHELLS,9952.0,7
184,97100,Raj,Vinnie,COCONUT,SEASHELLS,9952.0,7
...,...,...,...,...,...,...,...
1606,939700,Raj,Vinnie,COCONUT,SEASHELLS,10112.0,5
1633,950600,Raj,Vinnie,COCONUT,SEASHELLS,10110.0,6
1652,959400,Raj,Vinnie,COCONUT,SEASHELLS,10114.0,5
1688,986700,Raj,Vinnie,COCONUT,SEASHELLS,10126.0,9


Sells:


,timestamp,buyer,seller,symbol,currency,price,quantity
22,12500,Vinnie,Raj,COCONUT,SEASHELLS,10003.0,10
37,20100,Vinnie,Raj,COCONUT,SEASHELLS,9990.0,10
39,20800,Vinnie,Raj,COCONUT,SEASHELLS,9993.0,10
84,46400,Vinnie,Raj,COCONUT,SEASHELLS,9969.0,8
100,54100,Vinnie,Raj,COCONUT,SEASHELLS,9975.0,6
...,...,...,...,...,...,...,...
1636,951300,Vinnie,Raj,COCONUT,SEASHELLS,10104.0,8
1640,955400,Vinnie,Raj,COCONUT,SEASHELLS,10108.0,6
1675,978600,Vinnie,Raj,COCONUT,SEASHELLS,10129.0,10
1678,979600,Vinnie,Raj,COCONUT,SEASHELLS,10132.0,5



Trader: Valentina, Symbol: COCONUT_COUPON
Buys:


...

,timestamp,buyer,seller,symbol,currency,price,quantity
1,900,Valentina,Vinnie,COCONUT_COUPON,SEASHELLS,634.0,17
3,1900,Valentina,Vinnie,COCONUT_COUPON,SEASHELLS,635.0,16
4,2000,Valentina,Vinnie,COCONUT_COUPON,SEASHELLS,637.0,14
6,3900,Vinnie,Vinnie,COCONUT_COUPON,SEASHELLS,635.0,2
7,3900,Vladimir,Vinnie,COCONUT_COUPON,SEASHELLS,635.0,10
...,...,...,...,...,...,...,...
1703,996100,Valentina,Vinnie,COCONUT_COUPON,SEASHELLS,670.0,19
1706,996700,Valentina,Vinnie,COCONUT_COUPON,SEASHELLS,670.0,19
1708,997300,Valentina,Vinnie,COCONUT_COUPON,SEASHELLS,672.0,19
1709,997500,Valentina,Vinnie,COCONUT_COUPON,SEASHELLS,675.0,16



Trader: Vinnie, Symbol: COCONUT
Buys:


,timestamp,buyer,seller,symbol,currency,price,quantity
12,5200,Vinnie,Vinnie,COCONUT,SEASHELLS,9997.0,8
21,10900,Vinnie,Vinnie,COCONUT,SEASHELLS,10007.0,10
22,12500,Vinnie,Raj,COCONUT,SEASHELLS,10003.0,10
26,14900,Vinnie,Vinnie,COCONUT,SEASHELLS,10004.0,10
30,16000,Vinnie,Vinnie,COCONUT,SEASHELLS,10001.0,8
...,...,...,...,...,...,...,...
1696,992300,Vinnie,Vinnie,COCONUT,SEASHELLS,10126.0,5
1701,992900,Vinnie,Vinnie,COCONUT,SEASHELLS,10120.0,6
1704,996600,Vinnie,Vinnie,COCONUT,SEASHELLS,10127.0,9
1707,997100,Vinnie,Vinnie,COCONUT,SEASHELLS,10126.0,6


Sells:


,timestamp,buyer,seller,symbol,currency,price,quantity
12,5200,Vinnie,Vinnie,COCONUT,SEASHELLS,9997.0,8
18,9300,Raj,Vinnie,COCONUT,SEASHELLS,10005.0,6
21,10900,Vinnie,Vinnie,COCONUT,SEASHELLS,10007.0,10
26,14900,Vinnie,Vinnie,COCONUT,SEASHELLS,10004.0,10
30,16000,Vinnie,Vinnie,COCONUT,SEASHELLS,10001.0,8
...,...,...,...,...,...,...,...
1701,992900,Vinnie,Vinnie,COCONUT,SEASHELLS,10120.0,6
1704,996600,Vinnie,Vinnie,COCONUT,SEASHELLS,10127.0,9
1705,996600,Raj,Vinnie,COCONUT,SEASHELLS,10128.0,9
1707,997100,Vinnie,Vinnie,COCONUT,SEASHELLS,10126.0,6
